In [1]:
%matplotlib inline
from pynq.pmods import PMOD_ADC
from pynq.board import Switch
import time
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal


#functions

def Varprecision (index):
    rangein = precisione
    rangefin = precisione
    
    if (index < precisione):
        rangein = index
        
    if (index > finearray - precisione):
        rangefin = index
        
    return rangein,rangefin



def maxarray (candidatomax,b):
    
    if (candidatomax < max(b)):
        return 0
    
    else: 
        return 1

    

def minarray (candidatomin,b):
    
    if (candidatomin > min(b)):
        return 0
    
    else: 
        return 1

    
    
def Media (a):
    
    media=sum(a)/len(a)
    
    return media



def Contaeventi (a):
    mn=0
    mx=0
    contamin=0
    contamax=0
    
    for i in range (inizioarray,finearray):
        
        rangein,rangefin = Varprecision(i)
        
        mx = maxarray (a[i],a[i-rangein:i+rangefin])
        if (mx and a[i]>media):
            contamax = contamax + 1
            continue

        
        mn = minarray (a[i],a[i-rangein:i+rangefin])
        if (mn and a[i]<media):
            contamin = contamin + 1
    
    return contamin,contamax



def Durata_fasi (A_respiri):
    sonnoleggero=0
    sonnoprofondo=0
    tempoREM=0
    fasiintermedie=0

   
    for i in A_respiri:
        
        if (i >= sogliabassa_sonnoleggero and i <= sogliaalta_sonnoleggero):
            sonnoleggero=sonnoleggero+1
            
        elif (i >= sogliabassa_sonnoprofondo and i <= sogliaalta_sonnoprofondo):
            sonnoprofondo=sonnoprofondo+1
            
        elif (i >= sogliabassa_faseREM and i <= sogliaalta_faseREM):
            tempoREM=tempoREM+1
        
        else:
            fasiintermedie=fasiintermedie+1
            
    return sonnoleggero,sonnoprofondo, tempoREM, fasiintermedie



def Analisi (A_respiri):
    
    fasisonno=list()
    i=0
    
    while i < len(A_respiri):
            
        if (A_respiri[i] >= sogliabassa_faseREM and A_respiri[i] <= sogliaalta_faseREM):
            j = Lenght_REM( A_respiri[i:] )  
            
            if (j != 0):
                fasisonno.append( [i, i + j, j + 1, "REM"])
                i = i + j
        
        
        elif (A_respiri[i] >= sogliabassa_sonnoleggero and A_respiri[i] <= sogliaalta_sonnoleggero):
            j = Lenght_SL( A_respiri[i:] )  
            
            if (j != 0):
                fasisonno.append( [i, i + j, j + 1, "SL"])
                i = i + j
                
                            
        elif (A_respiri[i] >= sogliabassa_sonnoprofondo and A_respiri[i] <= sogliaalta_sonnoprofondo):
            j = Lenght_SP( A_respiri[i:] )  
            
            if (j != 0):
                fasisonno.append( [i, i + j, j + 1, "SP"])
                i = i + j
        
        
        
        i= i + 1
        
    return fasisonno

def REM_analisirec (A_respiri):
    i=0
    j=0
    
    while i < len(A_respiri):
        
        if (A_respiri[i] >= sogliabassa_faseREM and A_respiri[i] <= sogliaalta_faseREM):
            
            j = Lenght_REM( A_respiri[i:] )  
            
            if (j != 0):
                i = i + j
                return i
        
        i= i + 1
    
    


def Lenght_REM (Respiri_sliced):
    
    for i in range (len(Respiri_sliced)):
        
        if not Respiri_sliced[i] >= sogliabassa_faseREM:
            
            succ = [Respiri_sliced[i + x] for x in range (4) if (i+x < len(Respiri_sliced))]
            
            if i==0:
                return 0
            
            elif max(succ) < sogliabassa_faseREM:
                return i - 1
    
            else: 
                
                i = REM_analisirec(Respiri_sliced[i:]) + i
                
                return i
            
    else:
        return i     

def Lenght_SL (Respiri_sliced):
    
    for i in range (len(Respiri_sliced)):
        
        if ( Respiri_sliced[i] > sogliaalta_sonnoleggero or Respiri_sliced[i] < sogliabassa_sonnoleggero): ###attenzione!
            
            if (i > 4):
                return i - 1
    
            else: 
                return 0
        
    else:
        return i    
    
    

def Lenght_SP (Respiri_sliced):
    
    for i in range (len(Respiri_sliced)):
        
        if ( Respiri_sliced[i] > sogliaalta_sonnoprofondo ):
            
            if (i > 4):
                return i - 1
    
            else: 
                return 0
        
    else:
        return i    

    
    
def Array_grafico (fasi_sonno):
    hypnogram=list()
    
    for i in range (len(fasi_sonno)):
        
        if (fasi_sonno[i][3] == "REM"):
            
            for x in range (fasi_sonno[i][2]):
                hypnogram.append(3)
        
        elif (fasi_sonno[i][3] == "SL"):
            
            for x in range (fasi_sonno[i][2]):
                hypnogram.append(2)
            
        elif (fasi_sonno[i][3] == "SP"):
            
            for x in range (fasi_sonno[i][2]):
                hypnogram.append(1)
            
    return hypnogram



def Tempo_REM_print (fasi_sonno,durata_sonno):
    
    h = int (durata_sonno / 3600)
    m = int ( (durata_sonno % 3600) /60 ) 
    
    print ("Hai dormito per " + str(h) + " ore e " + str(m) + " minuti")
    
    fasi_REM = [fasi_sonno[x] for x in range (len(fasi_sonno)) if fasi_sonno[x][3] == "REM"]
    n_REM = len(fasi_REM)
    
    if n_REM>0:
    
        print ("Hai avuto la 1 fase REM di durata " + str (fasi_REM[0][2]) + " minuti a distanza di " 
                + str (fasi_REM[0][0] + 1) +  " minuti dall'inizio del sonno")
    
    else: print("Non hai avuto fasi REM")
    
    if n_REM>1:
    
        for i in range (1,n_REM):
        
            distanza_REM = fasi_REM[i][0] - fasi_REM[i-1][1]
            tempo_REM = fasi_REM[i][2]
            
            print ("Hai avuto la " + str(i + 1) + " fase REM di durata " + str(tempo_REM) + " minuti a " 
                   + str (distanza_REM) + " minuti di distanza dalla precedente" )
            
        
        
#define list of average breath per minute
Respiri=list()

#define constants of sleep analysis
sogliabassa_sonnoleggero = 12
sogliaalta_sonnoleggero = 18
sogliabassa_sonnoprofondo = 3
sogliaalta_sonnoprofondo = 4
sogliabassa_faseREM = 24
sogliaalta_faseREM = 36

#define sampling and filter constants
sampling_rate=10.0
nyq_rate=sampling_rate/2.0
lowpass=0.5
N = 101

#design lowpass filter
taps = signal.firwin(N, lowpass/nyq_rate)

#the following constant quantifies how many positions of the array before and after the value of interest
#are taken in consideration for the max-min research
precisione = 10 


#define the variable switch, used to start the sleep cycle and the analysis
switch=Switch(0)

while not switch.read():
    
    time.sleep(10)
    
inizio_sonno=time.time()

while switch.read():
    
    # initialize ADC on PMOD 1 (JB).
    # The ADC DA4 has 4 input channels at 12 bit resolution, however only the first 3: 
    # V1, V2 and V3 (form 0 to 2) can be used.
    # PMOD mapping is: PMOD 1 -> JB, PMOD 2 -> JC, PMOD 3 -> JD, PMOD 4 -> JE
    pmod_adc = PMOD_ADC(1)

    # read the tension value for V1 (V1 -> 0)
    value = pmod_adc.read(0) #???

    # start logging on V3 (V3 -> 2) with log interval of 100 ms
    # NOTE: the current driver allows to log only one input at a time,
    # to get data from multiple inputs at the same time you need to get them manually within a loop
    pmod_adc.start_log(0, 100)

    # wait for 60 second...
    time.sleep(60)

    # get and stop logging
    values = pmod_adc.get_log()
    avalues = np.array(values)

    #apply lowpass filter
    filtered_x = signal.filtfilt(taps, 0.5, avalues)
    
    #define useful variables linked to filtered_x
    inizioarray = 0 
    finearray = len(filtered_x)  
    media = Media(filtered_x)

    numero_minimi,numero_massimi = Contaeventi(filtered_x)
    
    Respiri.append(numero_massimi)

fine_sonno= time.time()

durata_sonno = int(fine_sonno - inizio_sonno)

#sleep analysis
A_respiri=np.array(Respiri)

fasi_sonno= Analisi (A_respiri)

Tempo_REM_print = (fasi_sonno,durata_sonno)

t=np.arange(len(A_respiri))/60
t1=[0,len(A_respiri)/60]
Rpm_x = [ x/60 for x in range (len(A_respiri)) if (x % 30 == 0) ]
                    
plt.figure(1)
plt.plot( t, A_respiri, 'r')
plt.title("Frequenza respiri/min")
plt.xlabel("Tempo")
plt.ylabel("Frequenza")
linea1, = plt.plot( t1, [sogliabassa_sonnoleggero,sogliabassa_sonnoleggero],'b--')
linea2, = plt.plot( t1, [sogliaalta_sonnoleggero,sogliaalta_sonnoleggero],'b--')
linea3, = plt.plot( t1, [sogliabassa_sonnoprofondo,sogliabassa_sonnoprofondo],'g--')
linea4, = plt.plot( t1, [sogliaalta_sonnoprofondo,sogliaalta_sonnoprofondo],'g--')
linea5, = plt.plot( t1, [sogliabassa_faseREM,sogliabassa_faseREM],'k--')
linea6, = plt.plot( t1, [sogliaalta_faseREM,sogliaalta_faseREM],'k--')
plt.legend([linea5, linea1, linea3], ['fase REM', 'sonno leggero', 'sonno profondo'])
plt.gca().axes.get_xaxis().set_ticks(Rpm_x)
plt.show ()


hypno = Array_grafico(fREM)
hypno_x = np.arange(len(hypno))
hypno_y=[0,1,2,3,4]
my_ticks=[" ", "Sonno profondo", "Sonno leggero", "Fase REM"]

plt.figure(2)
plt.plot (hypno_x, hypno)
plt.axis ([0,len(hypno),0,4])
plt.title ("ipnogramma")
plt.yticks(hypno_y, my_ticks)
plt.gca().axes.get_xaxis().set_visible(False)
plt.grid ()
plt.show ()

ImportError: No module named 'pynq'